# Objective of this practical session

## Why unsupervised learning?

In the last sessions we learned to handle labelled dataset properly and answer simple regression/classification problems, with a simple procedure:
    1. Cleaning/Data preparation: remove missing values
    2. Feature Selection/Feature Extraction with adapted statistics and     visualization tools
    3. Choice of some appropriate families models to test
    4. Parameter tuning for each kind of model
    5. Back to step 2) if the best models are not satisfying


However, you've seen in class that in many real world applications the dataset does not come with proper classes and the labelling in itself is the core of the problem. For example in finance one may want to define market regimes (are we in a crisis or in a financial bubble?) or to perform asset segmentation according to some properties of the history of the returns of each asset (identify trend following assets, pairs, etc...). 

In practice most of the machine learning models used in market finance are unsupervised models due to the lack of "ground truth" when looking at the current situation or forward.

## Content of this notebook

We recall the unsupervised classification framework: starting from samples $(X_1,\dots X_n)$, where for $i\in \{1,\dots n\}$, $X_i$ can be a vector of quantitative or qualitative variables, we will try to define a partition of $\{1,\dots n\}$ where individuals in the same element of the partition are "close" in some sense we need to define.
<br><br> 

Due to the lack of ground truth in real world datasets we will mainly work with simulated data, in order to understand the behaviour of each algorithm in our examples. <br><br> 


Based on these simulated data, we will perform the two simples unsupervised algorithms: $k$-means and hierarchical clustering. An optional section will allow students who want to go further to experiment Spectral clustering, which shows that more sophisticated methods exist. Our objective will be to understand for which kind of data each method can work, and why. <br><br> 



At the hand on the notebook we will work with a legend of Machine Learning: the MNIST dataset. https://fr.wikipedia.org/wiki/Base_de_donn%C3%A9es_MNIST
<br>


This dataset contains pictures of handwritten digits, and our objective will be to predict this number with an unsupervised model. We will figure that even though this method is far from the state of the art from this dataset a simple PCA does quite a good job in this problem. 

<br><br>

**Note:** this Notebook is yours and you are warmly encouraged to play with different settings, write some comments, etc.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Why-unsupervised-learning?" data-toc-modified-id="Why-unsupervised-learning?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Why unsupervised learning?</a></span></li><li><span><a href="#Content-of-this-notebook" data-toc-modified-id="Content-of-this-notebook-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Content of this notebook</a></span></li><li><span><a href="#How-to-generate-your-toy-example" data-toc-modified-id="How-to-generate-your-toy-example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>How to generate your toy example</a></span><ul class="toc-item"><li><span><a href="#k-Means" data-toc-modified-id="k-Means-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>k-Means</a></span><ul class="toc-item"><li><span><a href="#Visualization-of-k-Means-step-by-step" data-toc-modified-id="Visualization-of-k-Means-step-by-step-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Visualization of k-Means step-by-step</a></span><ul class="toc-item"><li><span><a href="#Mesh-the-input-space-to-display-the-decision-boundary" data-toc-modified-id="Mesh-the-input-space-to-display-the-decision-boundary-3.1.1.1"><span class="toc-item-num">3.1.1.1&nbsp;&nbsp;</span>Mesh the input space to display the decision boundary</a></span></li></ul></li><li><span><a href="#evaluation-Rand-index-vs-ARI-(Adjusted-Rand-Index)" data-toc-modified-id="evaluation-Rand-index-vs-ARI-(Adjusted-Rand-Index)-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>evaluation Rand index vs ARI (Adjusted Rand Index)</a></span></li><li><span><a href="#optimization-of-the-number-of-clusters" data-toc-modified-id="optimization-of-the-number-of-clusters-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>optimization of the number of clusters</a></span></li></ul></li><li><span><a href="#Limites-of-k-means-algorithm" data-toc-modified-id="Limites-of-k-means-algorithm-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Limites of k-means algorithm</a></span></li></ul></li><li><span><a href="#Hierarchical-clustering" data-toc-modified-id="Hierarchical-clustering-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Hierarchical clustering</a></span><ul class="toc-item"><li><span><a href="#Visualize-dendogram" data-toc-modified-id="Visualize-dendogram-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Visualize dendogram</a></span></li></ul></li><li><span><a href="#Spectral-clustering-(Facultative-part,-Bonus)" data-toc-modified-id="Spectral-clustering-(Facultative-part,-Bonus)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Spectral clustering (Facultative part, Bonus)</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Create-the-similarity-graph" data-toc-modified-id="Create-the-similarity-graph-5.0.1"><span class="toc-item-num">5.0.1&nbsp;&nbsp;</span>Create the similarity graph</a></span></li></ul></li></ul></li><li><span><a href="#PCA" data-toc-modified-id="PCA-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>PCA</a></span><ul class="toc-item"><li><span><a href="#MNIST" data-toc-modified-id="MNIST-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>MNIST</a></span></li></ul></li><li><span><a href="#Dimensionality-reduction-with-PCA" data-toc-modified-id="Dimensionality-reduction-with-PCA-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Dimensionality reduction with PCA</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Visualization-of-some-handwritten-digits" data-toc-modified-id="Visualization-of-some-handwritten-digits-7.0.1"><span class="toc-item-num">7.0.1&nbsp;&nbsp;</span>Visualization of some handwritten digits</a></span></li></ul></li></ul></li></ul></div>

In [1]:
%config InlineBackend.figure_format ='retina'

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# To generate you favorite toy example
from sklearn.datasets import make_blobs, make_moons

# Algorithms of the day
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics.cluster import adjusted_rand_score

# k-Means
from sklearn.cluster import KMeans

# Hierarchical clustering
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

# Spectral clustering with sklearn
from sklearn.cluster import SpectralClustering
# Spectral clustering by hand
from sklearn.metrics.pairwise import rbf_kernel  # similarity metric
import networkx as nx  # to play with graphs

# PCA
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits, load_iris
from sklearn.preprocessing import scale

## How to generate your toy example

`sklearn` allows you to build your favorite toy examples, click [here](https://scikit-learn.org/stable/modules/classes.html#samples-generator) for more details.

In particular, if you wish to generate a mixture of isotropic gaussians you can use the `make_blobs` methods.

Using the code from the last Python session, generate 300 independent Gaussian random vectors of $\mathbb{R}^2$: 100 with expectation $(0,0)$, 100 with expectation $(-4,4)$ and 100 with expectation $(4,4)$, and all with covariance matrix equal to indentity. Plot the simulations.

### k-Means

#### Visualization of k-Means step-by-step

For the initialization step use the following `class_inertia` function to

- associate match a point to the nearest centroid
- compute the total inertia of the initial configuration 

In [ ]:
def class_inertia(X, centroids):
    """Compute the square distance of each data points to the different centroids and 
    return the indices of the corresponding clusters and the total inertia.
    """
    
    sq_dist = np.array([np.einsum('ij,ij->i', X - c, X - c) for c in centroids])
    class_indices = np.argmin(sq_dist, axis=0)
    inertia = np.sum(sq_dist[class_indices, range(sq_dist.shape[1])])
    
    return class_indices, inertia

##### Mesh the input space to display the decision boundary

In [ ]:
# Useful for visualization
x_min, x_max = X_toy[:, 0].min() - .5, X_toy[:, 0].max() + .5
y_min, y_max = X_toy[:, 1].min() - .5, X_toy[:, 1].max() + .5

x1, x2 = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))
X_mesh = np.column_stack([x1.ravel(), x2.ravel()])

1. Read and Explain the following code
2. Run it and comment the results
3. In new cells change the number of clusters and comment on the results. Remember that in a real world problem you don't know the real number of cluster. Would you have been able to identify that the real number of clusters is 3 by looking at the partition? Why? If not, what number of clusters would have looked rationnal to you? (open question, no "true" answer, explain your reasonning).

In [4]:
n_clust = 3

# Initial centroids
centroids = np.column_stack([x_min + (x_max - x_min)*np.random.rand(n_clust),
                             y_min + (y_max - y_min)*np.random.rand(n_clust)])
# Initial inertia
_, inertia = class_inertia(X_toy, centroids)
y_mesh, _ = class_inertia(X_mesh, centroids)

for step in range(7):
    
    fig, ax = plt.subplots()
    
    plt.title('{} clusters, inertia = {:.2f}\niteration {}'.format(n_clust,inertia, step))
    # Display the points
    ax.scatter(X_toy[:,0], X_toy[:,1], c=cols[y_toy])
    # Display current centroids
    ax.scatter(centroids[:, 0], centroids[:, 1],
               marker='x', s=169, linewidths=3,
               color='w', zorder=10)
    # Display current decision boundary
    ax.contourf(x1, x2, y_mesh.reshape(x1.shape),
                alpha=0.4,
                cmap=plt.cm.Paired)

    plt.show()
    
    # Make a one step of k-Means starting from current centroids
    kmeans = KMeans(n_clusters=n_clust,
                    init=centroids,
                    max_iter=1,
                    n_init=1)
    # Fit the model
    kmeans.fit(X_toy)
    # Extract new centroids and inertia
    centroids = kmeans.cluster_centers_
    inertia = kmeans.inertia_
    
    # Predict the mesh
    y_mesh = kmeans.predict(X_mesh)

()

In [ ]:
# Cell for experiment with n_cluster ther than 3

In [ ]:
# Cell for experiment with n_cluster ther than 3

*Write your comment in this cell*



#### evaluation Rand index vs ARI (Adjusted Rand Index)

Use your $k$-means classifier to predict the classes. Compute the confusion matrix and the ARI.

*Note: you don't need to implement anything, use the sklearn functions*

In [5]:
# Answer here

#### optimization of the number of clusters

Elbow rule: plot the inertia as a function of the number of clusters. Conclude.
<br><br> 
*Indice: again, you don't have to compute it. Look at the attributes of your predictor*

In [ ]:
# Code Answer here

*Write your conclusion here*




### Limites of k-means algorithm

Your conclusions so far should be that the k-means algorithm performs pretty well for our toy dataset build with a gaussian mixture. Intuitively, it is clear that by construction points from the same cluster will be "close" in a $\mathbb{R}^2$ euclidian distance sense. 

<br> However, this is not always the case and even in $\mathbb{R}^2$ you could have very different structures that could be more challenging for k-means.

<br>In this section we will build new toy datasets and observe the behaviour of k-means on their structure.

Build two new toy datasets with `make_moons` and `make_blobs`. Low effort here, use the default parameters of the sklearn functions. Plot the datasets as in the first section. Comment on the shape on the clusters, how could you characterize them?

In [6]:
### Generate moons dataset

In [7]:
### Generate circles dataset

*Comment here on the charateristics of the clusters in each dataset*

Copy the code from the previous parts to run K-Means on these new datasets. Comment on the results: are they satisfying? Why? If not, is it *possible* to make them satisfying by transforming the data? How? (do not implement it) 

In [8]:
### K-means on moons dataset

In [9]:
### K-means on circle dataset

*Write your answers here*




## Hierarchical clustering

Recall how the method recursively clusters the population. Describe the different types of distances that can be chosen for merging existing clusters (you can use sklearn documentation).

* Answer here *




1. Use `AgglomerativeClustering` to cluster the toy dataset.
2. Display the corresponding clustering.
3. Investigate the different parameters of `AgglomerativeClustering` in particular the `linkage` argument.

In [ ]:
### Agglomerative clustering and displays

In [ ]:
### Other cells for parameters investigation

### Visualize dendogram

Compute the dendogram for the hierarchical clustering.

In [ ]:
Z = linkage(X_toy, method='ward')

plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index or (cluster size)')
plt.ylabel('distance')

dendrogram(Z,
           truncate_mode='lastp',  # show only the last p merged clusters
           p=3,  # show only the last p merged clusters
           leaf_font_size=12,
           show_contracted=True)

plt.show()

In [ ]:
y_pred = fcluster(Z, 3, criterion='maxclust') - 1

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(X_toy[:,0], X_toy[:,1], c=cols[y_pred], cmap='prism')  # plot points with cluster dependent colors
plt.show()

*** Warning : the following section is facultative, but the sections following it are not. You are free to skip this one but don't forget to go to the next ones***

## Spectral clustering (Facultative part, Bonus)

Consider the affinity
$K_{ij} \propto \exp(-\gamma \|x_i - x_j\|^2)$ 
between all pairs of points.

1. Use `rbf_kernel` to compute the corresponding `affinity_matrix` using 
2. Display the affinity matrix using `plt.matshow`

In [ ]:
affinity_matrix = rbf_kernel(X_toy, gamma=1/n_feat)
plt.matshow(affinity_matrix)
plt.colorbar()

#### Create the similarity graph

1. Create the `adjacency` matrix of the similarity graph defined by
$A_{ij} = \mathbb{1}_{K_{ij} > \epsilon}$, for a suitable $\epsilon$.

2. What does this correspond to ? 
3. Build and display the corresponding similarity graph using the following code

In [ ]:
eps = 0.1
adjacency = affinity_matrix > eps
plt.matshow(adjacency)

In [ ]:
G = nx.from_numpy_matrix(A=adjacency)
nx.draw(G, pos=X_toy,
        node_size=1, node_color=cols[y_toy],
        width=0.05, edge_color="grey")

1. Construct the `laplacian` matrix of the graph defined by
$L = D - A$,
where
- $A$ is the adjacency matrix computed previously
- $D$ is the diagonal degree matrix i.e. $D_{ii}=\sum_{j=1}^N A_{ij}$

$L$ is real symmetric, therefore can be diagonalized using `np.linalg.eigh`

2. Record the eigenvalues `e_vals` and eigenvectors `e_vecs` and

- plot the eigenvalues
- plot the 3 first eigenvectors $e_0, e_1, e_2$

3. What do you observe ?

In [ ]:
diag_degrees = np.diag(adjacency.sum(axis=0))
laplacian = diag_degrees - adjacency
plt.matshow(laplacian)
plt.colorbar()

Compute the eigenvalues and eigenvectors of the Laplacian matrix, and plot them.

In [ ]:
e_vals, e_vecs = np.linalg.eigh(laplacian)

In [ ]:
plt.plot(e_vals)

In [ ]:
fig, ax = plt.subplots()
for i in range(3):
    ax.plot(e_vecs[:, i], label=r'$e_{}$'.format(i))
plt.legend()

Use these results to perform dimension reduction: perform $k$-means clustering based on the projections of the points on the first axes corresponding to the smallest eigenvalues. 

Now apply the `SpectralClustering` method implemented by `sklearn`

In [ ]:
sp = SpectralClustering(n_clusters=3, affinity='rbf', gamma=1/n_feat)
y_pred = sp.fit_predict(X_toy)

plt.scatter(X_toy[:,0], X_toy[:,1], c=cols[y_pred])

## PCA

### MNIST

From [sklearn documentation](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html#sphx-glr-auto-examples-cluster-plot-kmeans-digits-py)

Load the MNIST dataset

In [ ]:
digits = load_digits()
X_mnist = scale(digits.data)
n_samples, n_features = X_mnist.shape

y_mnist = digits.target
n_digits = len(np.unique(y_mnist))

cols_mnist = np.array(plt.rcParams['axes.prop_cycle'].by_key()['color'])

What is the size of an image in this dataset ?

* Type your answer here *




## Dimensionality reduction with PCA

1. How many `n_components` of `PCA` must be kept to capture 95% of the explained variance ?

**Hint** Have a look at the documentation of `PCA` and use `.explained_variance_ratio_` and `np.cumsum`.

2. Make a suitable plot to visualize it.
3. Instanciate a `PCA` with the corresponding `n_components`
4. Project the data onto the corresponding lower dimensional subspace using `.transform()` and `.inverse_transform()` methods and display 15 of these approximated images.
5. Apply `PCA(n_components=2)` and display the points together with their class.
6. Based on this 2D embedding of the data, apply `KMeans` to cluster the points.
7. Evaluate this clustering

In [ ]:
cutoff = 0.95

pca = PCA(n_components=cutoff, svd_solver= 'full').fit(X_mnist)

var_ratio = pca.explained_variance_ratio_
plt.plot(np.cumsum(var_ratio))
plt.hlines(y=cutoff, xmin=0, xmax=pca.explained_variance_.size, label=cutoff)

plt.legend()

* Answer question 1) *




In [ ]:
var_ratio = pca.explained_variance_ratio_

fig, ax = plt.subplots()
plt.title('Cumulative explained variance = f(#components)')
ax.plot(np.cumsum(var_ratio))

cutoff = 0.95
ax.hlines(y=cutoff, xmin=0, xmax=var_ratio.size, label=cutoff)

n_comp_95 = np.where(np.cumsum(var_ratio) > 0.95)[0][0]
ax.vlines(x=n_comp_95, ymin=0, ymax=1)

plt.xlabel('#components')
plt.legend(loc='lower right')

In [ ]:
### Define Here the PCA model, transform and inverse transform of the dataset

# pca = ...
# X_trans = ...
# X_inv_trans = ...

### Use the following code to plot the inverse transform after filling
# the previous lines

fig, axes = plt.subplots(3, 5, figsize=(8,4))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_inv_trans[i].reshape(img_size),
              cmap=plt.cm.gray_r, interpolation='nearest',
              clim=(0,16))
    ax.set(xticks=[], yticks=[],
           xlabel='True label: {}'.format(y_mnist[ind_to_show[i]]))

Comment on the inverse transform of the digits you plotted, how satisfying is it?

* Answer*




To get good visualizations we are going to test kmeans with a transformed dataset with only two axis. Define below a PCA model with two components and fit it on the mnist dataset.

In [ ]:
# pca = 
# X_mnist_pca = ...

In [ ]:
# For the K-means part

x_min, x_max = X_mnist_pca[:, 0].min() - 1, X_mnist_pca[:, 0].max() + 1
y_min, y_max = X_mnist_pca[:, 1].min() - 1, X_mnist_pca[:, 1].max() + 1
x1, x2 = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))

X_mesh = np.column_stack([x1.ravel(), x2.ravel()])
y_mesh = kmeans.predict(X_mesh)

In [ ]:
### Define here  a K-Means model with the transformed MNIST dataset

# kmeans = ...
# kmeans.fit(X_mnist_pca)

In [ ]:
### Run this cell after filling the previous one 

fig, ax = plt.subplots(figsize=(12, 8))

# Display points
ax.scatter(X_mnist_pca[:,0], X_mnist_pca[:,1], s=2, c=cols_mnist[y_mnist])
# Add text label
for (xx, yy), lab in zip(X_mnist_pca, y_mnist):
    plt.text(xx, yy, s=str(lab))

# Display k-Means centroids
centroids = kmeans.cluster_centers_
ax.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
    
# Display boundary decision
ax.imshow(y_mesh.reshape(x1.shape), interpolation='nearest',
           extent=(x_min, x_max, y_min, y_max),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower', alpha=0.5)

Comment on the results. Does the spatial representation of the numbers make sense? Are some numbers easier to isolate than others, and does this make sense to you? What is the classification error? Plot the confusion matrix.

In [10]:
## Error and confusion matrix

* Answer *




#### Visualization of some handwritten digits

In [ ]:
images_and_labels = list(zip(digits.images, y_mnist))
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(2, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)